In [18]:
import requests, json
from zipfile import ZipFile
import pandas as pd
import numpy as np
import os
import tabula as tb
import camelot
import PyPDF2

In [ ]:
iteration = 3
with open("json_urls.txt","r") as t_op:
    for line in t_op.readlines():
        try:
            url = requests.get(line.replace("\n",""))
            text = url.text
            data = json.loads(text)
            out_file = f"train{iteration}.json"
            json.dump(data, out_file, indent = 4, ensure_ascii=False)
        except Exception as e:
            print(e)

In [10]:
file = "train6.zip"
with ZipFile(file, 'r') as zip:
    zip.extractall()

In [21]:
for file in os.listdir("train6"):
    data = PyPDF2.PdfFileReader("train6/"+file)
    #df = pd.DataFrame(data)

In [26]:
#print(data)
data.documentInfo
pageObj = data.getPage(0)
 
# extracting text from page

print(pageObj.extractText())


RE PORT  C ONT AMI N AC I ÓN
nº:
81596
FAC I LI T Y N AME:
M illerhill Re cy cling &  E nergy Rec o v ery  Ce ntre
81597
Fac i l i ty I ns pi r eI D:
UK.SEP A/2 0 0 0 0 2 6 5 1 .Facility
C OUNT RY :
United Kingdo m
C ONT I N EN T :
EUROP E
C I T Y:
M illerhill, D alkeith
EPR T RSec tor C ode:
5
epr tr Sec tor N ame:
Waste  and w aste wat er m anagem ent
Mai nAc ti v i ty C ode:
5 (b)
tar g etR eal as e:
AIR
pol l utant:
Carbo n diox ide (CO2 )
emi s s i ons :
1 ,3 2 E+ 0 8
DA Y:
4
MONT H:
12
YEA R:
2019
MET EOROLOGI C AL  C ONDI T I ONS
max_w i nd_s peed:
2 ,0 9 E+ 1 5
mi n_w i nd_s peed:
2 ,4 6 E+ 1 6
av g _w i nd_s peed:
2 ,0 7 E+ 1 5
max_temp:
5 ,7 4 E+ 1 5
mi n_temp:
9 ,4 2 E+ 1 5
av g _temp:
7 ,3 1 E+ 1 5
DA YS FOG:
6
RE PORT ER  N AME:
Jaso n M o ses
C I T Y_ I D
c6 6 2 b4 b4 d8 5 9 a9 c2 2 4 b5 ac0 acf2 2 1 7 4 8



AttributeError: 'PageObject' object has no attribute 'head'